# Assignment 1: Chinese character "detection"
LT2326, Autumn 2021

Name: Max Boholm (gusbohom)

# Introduction

This notebook cotains the code for *Assignment 1: Chinese character "detection"* for the course *Machine learning for statistical NLP: Advanced* (course code LT2326), Autumn 2021. The notebook is organized into the folowing parts:

*    Meta variables (the term *hyperparameter* is here reserved for decisions on the models), which define ... the loacation (path) of the data, ... 
*    Data preparation
*    Definition and training of two models
*    Testing and evaluation
*    ...




# TO DO

**THINK AGAIN ABOUT WHAT THE PROBLEM IS**
How do you go from the "topographic" representation back to coordinates?
Consider using coordinates as targets, how do you calculate the loss? (Perhaps not that hard... one output for every dimension of coordinates... but then: ) How to handle variable length of boxes in image?

???
predict coordinates
for a 2048 x 2048 vector translate those coordinates to 0s and 1s
compute loss for that vector


*    Rescaling - how stupid is my approach? WORKS FOR DEVELOPMENT
*    How to build (convolutional) models?
    *    Does I have to consider padding? (If so, perhaps best to use a `torch` dataloader) I THINK THE CONVOLUTIONAL LAYERS HELPS ME WITH THAT
*    How to visualize?
    *    Reverse flattening
    *    Heatmap simple
    *    Heatmap-layer
*    Evaluation metrics

# Libraries

In [1]:
import glob
import json
import random

from PIL import Image
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.path as mplpath

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Meta variables

In [2]:
#path = "../../scratch/lt2326-h21/a1/"
path = "../develop_util/"
images_dir = path + "images/"      # the directory of images to be data
meta_ctw   = path + "info.json"   # the (path to) the general meta file of CTW 
meta_train = path + "train.jsonl" # the (path to) the file containing the annotations of CTW training data

train_proportion = 0.7 # the proportion of training data; proportion of test data will be the complement of this number

#device = torch.device('cuda:0')
device = torch.device('cpu')

path_to_save_models = "../models/"

# For development purposes
restriction = None # set to an integer, if subsample of the data is to be used in e.g. the development phase
rescale_input_to = 50 # ...
rescale_output_to = 128 # ...

## Part 1: data preparation

*Decision:* batching is kept as list until feeding it to model (`flat_batch`). The list makes the evaluation part simpler as we do not have to bend our minds around multi-dimensional tensors. 

### Definitions of functions

In [3]:
def only_train(file_dir = images_dir, meta_file = meta_ctw):
    """Tests whether image files in a directory is part of the training set of the CTW dataset, 
       as defined by the json metafile for CTW. Returns a list of filenames such that they are both
       in the specified directory and the CTW training set.
    """
    meta = json.load(open(meta_file,"r"))
    train_files_CTW = [entry["file_name"] for entry in meta["train"]]
    files_to_keep = []
    potential_files=[file.split("/")[-1] for file in glob.glob(file_dir+"*.jpg")]
    
    for file in potential_files:
        if file in train_files_CTW:
            files_to_keep.append(file)
            
    return files_to_keep

def CTW_mapper(files, meta = meta_train):
    """ Identifies annotations for files from the training set of the CTW dataset. 
        Returns pyhoton dictionary that maps filenames (keys) with annotations (values), 
        which like in the original format is a list of lists of json elements / python dictinaries. 
    """
    mapping = {}
    with open(meta, "r") as f:
        annotations_data = [json.loads(line) for line in f.readlines()]
        for file in files:
            for annotation in annotations_data:
                if annotation["image_id"] == file[:-4]:
                    mapping[file] = annotation["annotations"]
                    break
    
    return mapping
                    
def img2array(file, directory = images_dir, rescale = rescale_input_to):
    """ Takes a filename of an image in a directory and returns an numpy array 
        corresponding to the image.
    """
    img = Image.open(directory+file)
    
    if rescale != None:
        img = img.resize((rescale, rescale))
    
    img_np = np.array(img)
    
    return img_np

def bbox2array(file, mapping, height  = 2048, width   = 2048, rescale = rescale_output_to): 
    """ Builds a matrix of 0s and 1s representing the bbox as 
        defined by the coordinates of in the CTW annotations.
    """
    
    if rescale != None:
        rescale_by = rescale / height # must come first
        height = rescale
        width = rescale
    
    char_matrix = np.zeros((height, width))
    
    char_areas = []
    for block in mapping[file]:
        for character in block:
            if character["is_chinese"] == True:
                xmin = int(character["adjusted_bbox"][0])
                ymin = int(character["adjusted_bbox"][1])
                w    = int(character["adjusted_bbox"][2])
                h    = int(character["adjusted_bbox"][3])
                
                if rescale != None:
                    xmin = int(xmin * rescale_by)
                    ymin = int(ymin * rescale_by)
                    w    = int(w    * rescale_by)
                    h    = int(h    * rescale_by)
                
                char_areas.append((xmin, ymin, w, h))
    
    for xmin, ymin, w, h in char_areas:
        r1 = height - ymin - h
        r2 = r1 + h
        c1 = width - xmin
        c2 = c1 + w
        
        char_matrix[r1:r2 , c1:c2] = 1
        
    return char_matrix

def polygon2array(file, mapping, height = 2048, width = 2048, rescale = rescale_output_to):
    """ Builds a matrix of 0s and 1s representing the character polygons as 
        defined by the coordinates of in the CTW annotations. 
    """
    
    polygons = []
    for block in mapping[file]: # mapping maps files with their annotations
        for character in block:
            if character["is_chinese"] == True:
                polygons.append(character["polygon"])
    
    if rescale != None:
        rescale_by = rescale / height # must come first
        height = rescale
        width = rescale
        polygons = [[[point * rescale_by for point in points] for points in set_of_points] for set_of_points in polygons]
    
    every_point = np.array([[[h,w] for h in list(range(height))] for w in list(range(width))]).reshape(height*width, 2)
    zeros_to_update = np.zeros(height * width)
    
    for polygon in polygons:
        path = mplpath.Path(np.array(polygon))
        hits = np.asarray(path.contains_points(every_point), int)
        zeros_to_update += hits
    
    matrix = zeros_to_update.reshape(height, width)
    
    return matrix

def data_builder(files, directory, mapping, restriction = restriction):
    """ Compiles the dataset for use. Returns a list of dictionaties, such that each
        element of the list contains:
        -  the filename; key: "file"
        -  a vectorized instance of the training data; key: "img_vector"
        -  a vectorized instance of the labels, or targets (a vector of 0s and 1s indicating 
           boxes of characters in images); key: "label"
        
        Note: (1) the format of instances (training input and targets) are numpy arrays; and
        (2) the instances have "matrix shape". For these reasons, the output of the data_builder()
        requires further processing for it to be ready for pytorch processing. The functions 
        standardizer() and numpy2torch() is does required further down the pipline of data
        preprocessing.
    """
    
    data = []
    for file in files: 
        instance = {}
        instance["file"] = file
        instance["img_vector"] = img2array(file)
        #instance["label"] = bbox2array(file, mapping)
        instance["label"] = polygon2array(file, mapping)
        data.append(instance) 
    
    if restriction != None:
        random.shuffle(data)
        data = data[:restriction]
        
    return data

def standardizer(dataset, scaler = StandardScaler()):
    """ Standardizes the image vectors of a dataset to z-scores using StandardScaler() 
        from the library sklearn.preprocessing. 
    """
    std_data = []
    #N = len(dataset)
    example = dataset[0]["img_vector"]
    x, y, z = example.shape
    n_features = example.size # ... or x * y * z
    
    for instance in dataset:
        std_data.append(instance["img_vector"].reshape(n_features))
    
    scaled_data = scaler.fit_transform(std_data)
    
    for i, scaled_ins in enumerate(scaled_data):
        dataset[i]["img_vector"] = scaled_ins.reshape(x, y, z)

def numpy2torch(dataset, device = device, permute = True):
    """ For a dataset, transforms its numpy arrays to torch tensors. If permute = True,
        image vectors are permuted such that ... 
    """
    
    #print("Shape of vector before: ", dataset[0]["img_vector"].shape)
    
    for instance in dataset:
        if permute == True: # ... hmmm 
            instance["img_vector"] = torch.Tensor(instance["img_vector"], device = device).permute(2,0,1)
        else:
            instance["img_vector"] = torch.Tensor(instance["img_vector"], device = device)
        instance["label"] = torch.Tensor(instance["label"], device = device)
    
    #print("Shape of vector after: ", dataset[0]["img_vector"].shape)


### Calling functions: creating the overall dataset

In [4]:
files = only_train()
mapping = CTW_mapper(files)
my_data = data_builder(files, images_dir, mapping)
standardizer(my_data)
numpy2torch(my_data)

### Save and load dataset

In [ ]:
# Save dataset as .jsonl

In [ ]:
#with open("dataset.jsonl", "r") as d:
#    my_data = [json.load(instance) for instance in d] # this will not handle np stuff I guess

### Splitting the dataset into train and test sets

In [5]:
def split(data       = my_data, 
          train_prop = train_proportion, 
          val_prop   = None):
    """ Splits a dataset into training data, testing data and, if selected,
        validation data. Note that the proportions of training data, test data
        and validation data (optional) must not exceed 100%. 
    """
    
    if val_prop != None:
        train_to_idx = int(len(data) * train_prop)
        val_to_idx   = int(len(data) * val_prop) + train_to_idx
        train = data[:train_to_idx]
        val   = data[train_to_idx:val_to_idx]
        test  = data[val_to_idx:]
        return train, val, test
    else:
        train_to_idx = int(len(data) * train_prop)
        train = data[:train_to_idx]
        test  = data[train_to_idx:]
        return train, test

train_set, test_set = split() #val?

### Defining a dataloader
In training, the function will be called with the `train_set` as argument in every iteration (epoch) yielding  randomized and batched traing inputs.

In [6]:
# do we need padding?! no part of convolutional layer

def dataloader(data, batch_size):
    """ Takes a (proportion of) a dataset and returns a randomized iterator 
        of the data organized into batches as defined by batch_size.
        
        Note: the dataloader preserves the "matrix shape" of trainingdata and 
        targets. Since pytorch neural networks require "flat" shapes of data
        the function flat_batch() is used to let data flow trough training
        in the desired format. 
    """
    
    random.shuffle(data)
    
    for group in [data[i : i+batch_size] for i in range(0, len(data), batch_size)]:
        files = []
        img_vecs = []
        labels = []

        for instance in group:
            files.append(instance["file"])
            img_vecs.append(instance["img_vector"])
            labels.append(instance["label"])

        batch = {"file":files, 
                 "img_vector":img_vecs, 
                 "label":labels}
    yield batch

### Checking

In [7]:
my_data[0]

{'file': '0000174.jpg',
 'img_vector': tensor([[[ 1.5574,  1.5627,  1.5741,  ...,  0.5990, -0.1580,  0.6819],
          [ 1.5664,  1.5732,  1.5761,  ..., -0.0143,  0.3681, -0.0704],
          [ 1.5640,  1.5757,  1.5463,  ...,  0.7814,  0.5509,  1.1346],
          ...,
          [-1.3626, -1.3501, -1.3124,  ...,  1.7152,  1.7382,  1.4885],
          [-1.4054, -1.3861, -1.2840,  ...,  1.6274,  1.8201,  1.7719],
          [-1.2637, -1.2589, -1.2508,  ...,  1.4272,  1.6324,  1.7605]],
 
         [[ 1.4955,  1.5454,  1.5554,  ...,  0.2724, -0.6234,  0.3421],
          [ 1.5248,  1.5440,  1.5131,  ..., -0.5830,  0.0598, -0.4879],
          [ 1.5086,  1.5072,  1.5112,  ...,  0.3654,  0.1018,  0.9168],
          ...,
          [-1.4167, -1.3837, -1.3778,  ...,  1.6411,  1.6386,  1.3897],
          [-1.4553, -1.4766, -1.3444,  ...,  1.5661,  1.7879,  1.7393],
          [-1.3110, -1.2970, -1.2832,  ...,  1.3103,  1.5133,  1.7198]],
 
         [[ 1.1902,  1.2638,  1.4043,  ..., -0.4131, -1.4313, 

## Part 2: the models

### General traing procedure

In [8]:
def flat_batch(batch):
    """Takes a python list of length B of more-than-one dimensional tensors (N, M, ...) and 
    returns a tensor of shape: (B, M*N*...)..."""
    
    return torch.stack([torch.flatten(instance) for instance in batch])

# do we need device assignation here?
    

In [14]:
def trainer(model, # Must be an instance of a model!
            name_of_model,
            learning_rate,
            epochs,
            batch_size,
            train_data = train_set,
            val_data = None,
            save_model = False,
            path_for_saving_model = path_to_save_models,
            my_optimizer = optim.Adam,
            my_loss_function = nn.BCELoss()):
    """ Specifices a general training procedure for a model. 
        Note: trainer() requires an instantiated model as model argument. 
    """
    
    optimizer = my_optimizer(model.parameters(), lr=learning_rate)    
    
    #model = my_model
    model.to(device)
    model.train()
    
    loss_function = my_loss_function
    
    #total_loss = 0
    
    for epoch in range(epochs):
        epoch_loss = 0
        
        iterator = dataloader(train_set, batch_size)
        for i, batch in enumerate(iterator):
            
            output = model(batch["img_vector"])
            targets = flat_batch(batch["label"])
            
            loss = loss_function(output, targets)
            
            #total_loss += loss.item()
            epoch_loss += loss.item()
            #print("Epoch: ", epoch+1, "Batch: ", i, "Total loss: ", total_loss/(i+1), end='\r')
            loss.backward() # compute gradients
            optimizer.step() # update parameters
            optimizer.zero_grad # reset gradients
            
        #print()
        print(f"Epoch: {epoch+1} (out of {epochs}); total loss: {epoch_loss}.")

            
        if val_data != None:
            model.eval()
            
            # DO SOMETHING
            
            model.train()
            
    if save_model == True:
        torch.save(model, path_for_saving_model+name_of_model+".pt")

### Model 0: Simple Model

This is a very simple model, basically passing data from input to output. It is **used for development reasons only**. I have kept it here as it might become useful for future development, but *it can be ignored for examination of Assignment 1*.

#### Model defintion

In [15]:
class SimpleModel(nn.Module):
    def __init__(self, inp, hidden, outp):  
        super(SimpleModel, self).__init__()
        
        self.layer1 = nn.Linear(inp, hidden)
        self.classifier = nn.Linear(hidden, outp)
  
    def forward(self, batch): 
        
        flat = flat_batch(batch)
        compression = F.relu(self.layer1(flat))
        output = torch.sigmoid(self.classifier(compression))
        
        return output

#### Training and saving model

In [16]:
shape_of_input = my_data[0]["img_vector"].shape
input_size = shape_of_input[0] * shape_of_input[1] * shape_of_input[2]
shape_of_output = my_data[0]["label"].shape
output_size = shape_of_output[0] * shape_of_output[1]

#print(shape_of_input)
#print(shape_of_output)

my_simple_model = SimpleModel(inp=input_size, hidden=100, outp=output_size)

In [17]:
my_simple_model.parameters

<bound method Module.parameters of SimpleModel(
  (layer1): Linear(in_features=7500, out_features=100, bias=True)
  (classifier): Linear(in_features=100, out_features=16384, bias=True)
)>

In [18]:
trainer(my_simple_model, # Must be an instance of a model!
        "simplemodel.pt",
        learning_rate=0.005,
        epochs=2,
        batch_size=2,
        save_model = True)

Epoch: 1 (out of 2); total loss: 0.695411205291748.
Epoch: 2 (out of 2); total loss: 0.7315812110900879.


### Model 1: Diabolo Autoencoder

#### Model definition

In [25]:
class DiaboloModel(nn.Module):
    def __init__(self, inp, outp):   
        super(DiaboloModel, self).__init__()
        
        self.input = inp
        self.output = outp
        
        self.compression1 = int(self.input / 2)
        self.compression2 = int(self.input / 4)
        self.compression3 = int(self.input / 8)
        self.compression4 = int(self.input / 16)
        
        self.dilation1 = int(self.output / 8)
        self.dilation2 = int(self.output / 4)
        self.dilation3 = int(self.output / 2)
        
        self.encoder = nn.Sequential(
            nn.Linear(self.input, self.compression1),
            nn.ReLU(),
            nn.Linear(self.compression1, self.compression2),
            nn.ReLU(),           
            nn.Linear(self.compression2, self.compression3),
            nn.ReLU(),
            nn.Linear(self.compression3, self.compression4),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(self.compression4, self.dilation1),
            #nn.Dropout(0.05),
            nn.ReLU(),
            nn.Linear(self.dilation1, self.dilation2),
            nn.ReLU(),            
            nn.Linear(self.dilation2, self.dilation3),
            nn.ReLU(),
            nn.Linear(self.dilation3, self.output),
            nn.Sigmoid()
        )
        
 
    def forward(self, batch): 
        
        flat = flat_batch(batch)
        encoded = self.encoder(flat)
        decoded = self.decoder(encoded)
        
        return decoded

#### Training and saving model

In [26]:
shape_of_input = my_data[0]["img_vector"].shape
input_size = shape_of_input[0] * shape_of_input[1] * shape_of_input[2]
shape_of_output = my_data[0]["label"].shape
output_size = shape_of_output[0] * shape_of_output[1]

#print(shape_of_input)
#print(shape_of_output)

my_diabolo_model = DiaboloModel(inp=input_size, outp=output_size)

In [27]:
my_diabolo_model.parameters

<bound method Module.parameters of DiaboloModel(
  (encoder): Sequential(
    (0): Linear(in_features=7500, out_features=3750, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3750, out_features=1875, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1875, out_features=937, bias=True)
    (5): ReLU()
    (6): Linear(in_features=937, out_features=468, bias=True)
    (7): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=468, out_features=2048, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2048, out_features=4096, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4096, out_features=8192, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8192, out_features=16384, bias=True)
    (7): Sigmoid()
  )
)>

In [28]:
nepochs = 2
batchsz = 2
name = "Diab_{}e{}b".format(nepochs, batchsz)

trainer(my_diabolo_model, # Must be an instance of a model!
        name,
        learning_rate=0.005,
        epochs=nepochs,
        batch_size=batchsz,
        save_model = True)

Epoch: 1 (out of 2); total loss: 0.6931681036949158.
Epoch: 2 (out of 2); total loss: 1.9287109375.


### Model 2.1: Convolutional Model

#### Model definition

In [30]:
class ConvModel(nn.Module):
    def __init__(self, inp_height, inp_width, outp):   
        super(ConvModel, self).__init__()
        
        self.height = inp_height
        self.width = inp_width
        self.getting_out = self.height * self.width * 3 
        # 3 for channels
        # ... plus operation to acount for pooling stuff
        
        self.output = outp
        self.dilation = self.output / 2

        
        self.convolution = nn.Sequential(
            Conv2d(3, 3, 3, stride = 1, padding=True), # 3 channels in (RGB), 3 out, window of 3x3
            nn.BatchNorm2d(3),
            nn.ReLU(),
            #nn.MaxPool2d(2, 2), # 2x2 window with stride of 2
        )
        
        self.out = nn.Sequential(
            nn.Linear(self.getting_out, self.dialation),
            nn.Dropout(0.05),
            nn.Tanh(),
            nn.Linear(self.dialation, self.output),
            nn.Sigmoid()
        )
        
 
    def forward(self, batch): 
        
        batch = torch.stack(batch) # device?
        features = self.convolution(batch)
        output = self.out(features)
        
        return output

#### Training and saving model

### Model 2.2: Convolutional Model with Pooling

#### Model definition

#### Training and saving model

## Part 3: testing and evaluation

### Evaluation Metrics

Two basic types of evaluation metrics are considered:

1. The "continious" ("analog") metric of *mean squared error*.
2. Threshold-based ("dialog", frequency-based) metrics, assuming a treshold *t* for a classfier *C*, such that for every pixel *x*, if the probaility predicted for *x* (i.e. *p(x)*) is greater than *t*, then *C(x)* = 1, if not, *C(x)* = 0. Represented by a threhold-classification, true positives (TP), false positives (FP), true negatives (TN) and false neagtives (FN) can be calculated and therfore also standard measures of *accuracy*, *recall*, *precision* and *F1*. 

Both types of metrics (analog and digital) can be measured for the model's performance on *individual* images. However, general measures of the model's performance on the *complete* test set must be considered. For this, two approaches are used:

*    A pooled approach: the evaluation metrics are calculated for the concatenation of predictions for every image of the test set in relation to the concatenation of every true label (pixel map of polygon boxes). 

`Metric([PredictionImage-1 + ... + PredictionImage-n], [TruthImage-1 + ... + TruthImage-n])` (where `+` here stands for concatenation, not addition). 
*    An averaging approach: taking the mean and standard deviation of a particular metric calculated for individual images 

`Mean([Metric(image-1), ..., Metric(image-n)])` 



    

In [ ]:
def th_metrics(tp, fp, tn, fn):
    """ Calculates Accuracy, Recall, Precision, and F1 from frequencies of 
        true postives (tp), false postives (fp), true negatives (tn), and
        false neagtives (fn).
    """
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = (2 * recall * precision) / (recall + precision)
    return accuracy, recall, precision, f1

In [ ]:
def mean(array):
    """ Calculates the mean and standard deviation of an aray of numbers.
    """
    
    mean = np.mean(array)
    std  = np.std(array)
    return mean, std

In [ ]:
class Evaluation:
    """ For storing and handling information from the evaluation of models.
    """
    
    def __init__(self, name):
        self.name = name
        self.pooled_mse = "Not yet defined"
        self.pooled_acc = "Not yet defined"
        self.pooled_rec = "Not yet defined"
        self.pooled_prc = "Not yet defined"
        self.pooled_f1  = "Not yet defined"
        self.mean_mse = ("Not yet defined", "Not yet defined")
        self.mean_acc = ("Not yet defined", "Not yet defined")
        self.mean_rec = ("Not yet defined", "Not yet defined")
        self.mean_prc = ("Not yet defined", "Not yet defined")
        self.mean_f1  = ("Not yet defined", "Not yet defined")
        self.summary  = "\n".join([f"Model {self.name} performs as follows:", 
                                   f"Pooled MSE: {self.pooled_mse}",
                                   f"Pooled Accuracy: {self.pooled_acc}",
                                   f"Pooled Recall: {self.pooled_rec}",
                                   f"Pooled Precision: {self.pooled_prc}",
                                   f"Pooled F1: {self.pooled_f1}",
                                   f"Mean MSE: {self.mean_mse[0]} (std = {self.mean_mse[1]})",
                                   f"Mean Accuracy: {self.mean_acc[0]} (std = {self.mean_acc[1]})",
                                   f"Mean Recall: {self.mean_rec[0]} (std = {self.mean_rec[1]})",
                                   f"Mean Precision: {self.mean_prc[0]} (std = {self.mean_prc[1]})",
                                   f"Mean F1: {self.mean_f1[0]} (std = {self.mean_f1[1]})"])
        

    
    def compare(self, evaluation_model1, evaluation_model2):
        pass
    
    def save(self, path, filename):
        
        with open(path+filename, "w") as e:
            e.write(self.summary)
    
    def summary(self):
        txt = self.summary
        print(txt)
    

In [ ]:
ev = Evaluation("mamma_mu")

In [ ]:
ev.summary


In [ ]:
string = "mamma\nmu"

In [ ]:
print(string)

### Testing: setting hyperparameters

In [ ]:
batch?
stride?
iterations?
window?

### Evaluation: performance of best models

*    Pooled metrics (concatenate prediction-1, ..., prediction-n (P) and label-1, ..., label-n (L) and calculate metric(P, L).
*    Means (with std)  
    

In [ ]:
def evaluator(model, threshold = 0.5):
    """ Defines a pipeline for evaluation by evaluation metrics.
    """
    model.eval()
    
    # evaluation = Evaluation(name)
    
    
    # mse_pooled = ... 1. make a list of inp_vecs / labels; 2. make a torch tensor; 3. feed to model; 4. "evaluate"
    # truth_pooled = ... 
    mse_calculated = []
    
    th_frequencies_pooled = {"tp": 0, "fp": 0, "tn": 0, "fn": 0}
    th_metrics_calc = {"accuracy": [], "recall": [], "precision": [], "f1": []}

    for instance in test_set:
        
        prediction = model(torch.flatten(instance["img_vector"]))
        truth = torch.flatten(instance["label"]).int()
        #print(truth)

        mse = F.mse_loss(prediction, truth)
        mse_calculated.append(mse.item())
        
        roundof = (prediction >= threshold).int()
        
        tp = sum(roundof * truth)
        #print("TP: ", tp)
        fp = sum(roundof * (~truth.bool()).float())
        #print("FP: ", fp)
        tn = sum((~roundof.bool()).float() * (~truth.bool()).float())
        #print("TN: ", tn)
        fn = sum((~roundof.bool()).float() * truth)
        #print("FN: ", fn)
        
        accuracy, recall, precision, f1 = th_metrics(tp, fp, tn, fn)
        
        for key, value in zip(["tp", "fp", "tn", "fn"], [tp.item(), fp.item(), tn.item(), fn.item()]):
            th_frequencies_pooled[key]+=value
        
        for key, value in zip(["accuracy", "recall", "precision", "f1"], [accuracy, recall, precision, f1]):
            th_metrics_calc[key].append(value)
            
    pooled_accuracy, pooled_recall, pooled_precision, pooled_f1 = th_metrics(
        th_frequencies_pooled["tp"], 
        th_frequencies_pooled["fp"], 
        th_frequencies_pooled["tn"], 
        th_frequencies_pooled["fn"])
    
    mean_mse, std_mse             = mean(mse_calculated)
    mean_accuracy, std_accuracy   = mean(th_metrics_calc["accuracy"])
    mean_recall, std_recall       = mean(th_metrics_calc["recall"])
    mean_precision, std_precision = mean(th_metrics_calc["precision"])
    mean_f1, std_f1               = mean(th_metrics_calc["f1"])
    
    print("Pooled MSE: ", "To Be Fixed")
    print("Pooled Accuracy: ", pooled_accuracy)
    print("Pooled Recall: ", pooled_recall)
    print("Pooled Precision: ", pooled_precision)
    print("Pooled F1: ", pooled_f1)
    print("Mean MSE = {0:.4} (std = {1:.4})".format(mean_mse, std_mse))
    print("Mean Accuracy = {0:.4} (std = {1:.4})".format(mean_accuracy, std_accuracy))
    print("Mean Recall = {0:.4} (std = {1:.4})".format(mean_recall, std_recall))
    print("Mean Precision = {0:.4} (std = {1:.4})".format(mean_precision, std_precision))
    print("Mean F1 = {0:.4} (std = {1:.4})".format(mean_f1, std_f1))
    
    # some text format output
    # keep track of bad examples


In [ ]:
evaluator(my_simple_model)

In [ ]:
evaluator(my_diabolo_model)

### Visualizations

In [ ]:
# show true box on image
# show predicted heatmap on image
# show true box on predicted heatmap